# FastAPI Basic Route Handling 🛣️

This notebook covers the fundamentals of creating routes in FastAPI.

**What you'll learn:**
1. **Creating GET Endpoints** - Retrieve data from your API
2. **Path Parameters** - Dynamic URL segments (`/users/{user_id}`)
3. **Query Parameters** - URL query strings (`?skip=0&limit=10`)
4. **Request Body with POST** - Sending data to create resources
5. **Return Types and Responses** - Controlling what your API returns

Let's master FastAPI route handling! 🚀

## Part 1: Creating GET Endpoints

GET endpoints are used to retrieve data. They're the most common type of endpoint.

### Basic GET Endpoint

The simplest GET endpoint returns data:

```python
@app.get("/")
def read_root():
    return {"message": "Hello, FastAPI!"}
```

### Key Points:
- ✅ Use `@app.get()` decorator
- ✅ Function name can be anything (descriptive is better)
- ✅ Return value is automatically converted to JSON
- ✅ GET requests are **safe** and **idempotent**

In [ ]:
# Example: Basic GET Endpoints

from fastapi import FastAPI

app = FastAPI()

# Simple GET endpoint
@app.get("/")
def read_root():
    """Root endpoint - returns welcome message"""
    return {"message": "Welcome to FastAPI!"}

# GET endpoint returning a list
@app.get("/items")
def list_items():
    """List all items"""
    return {
        "items": [
            {"id": 1, "name": "Item 1"},
            {"id": 2, "name": "Item 2"},
            {"id": 3, "name": "Item 3"}
        ]
    }

# GET endpoint with type hints
@app.get("/users")
def get_users() -> dict:
    """Get all users with type hint"""
    return {
        "users": [
            {"id": 1, "name": "Alice"},
            {"id": 2, "name": "Bob"}
        ]
    }

print("✅ GET endpoints created!")
print("📝 Available routes:")
print("   - GET /")
print("   - GET /items")
print("   - GET /users")
print("\n💡 FastAPI automatically:")
print("   - Converts return values to JSON")
print("   - Generates API documentation")
print("   - Validates types (if specified)")

## Part 2: Path Parameters

Path parameters allow you to capture values from the URL path.

### Syntax

```python
@app.get("/items/{item_id}")
def get_item(item_id: int):
    return {"item_id": item_id}
```

### How It Works:
- `{item_id}` in the URL path becomes a function parameter
- FastAPI automatically extracts and validates the value
- Type hints (`int`) ensure the value is converted and validated

### Path Parameter Rules:
1. ✅ Parameter name must match URL segment
2. ✅ Type hints enable automatic validation
3. ✅ Invalid types return 422 error automatically
4. ✅ Order matters - more specific routes first

In [ ]:
# Example: Path Parameters

from fastapi import FastAPI

app = FastAPI()

# Path parameter with int type
@app.get("/items/{item_id}")
def get_item(item_id: int):
    """
    Get item by ID
    
    - **item_id**: The ID of the item (must be an integer)
    """
    return {
        "item_id": item_id,
        "name": f"Item {item_id}",
        "type": type(item_id).__name__  # Shows it's an int
    }

# Path parameter with string type
@app.get("/users/{username}")
def get_user(username: str):
    """Get user by username"""
    return {
        "username": username,
        "message": f"User {username} found"
    }

# Multiple path parameters
@app.get("/users/{user_id}/posts/{post_id}")
def get_user_post(user_id: int, post_id: int):
    """Get a specific post by a specific user"""
    return {
        "user_id": user_id,
        "post_id": post_id,
        "content": f"Post {post_id} by user {user_id}"
    }

# Path parameter with type validation
@app.get("/products/{product_id}")
def get_product(product_id: int):
    """
    Get product by ID
    
    FastAPI will:
    - Convert string from URL to int
    - Return 422 error if not a valid integer
    - Validate automatically
    """
    if product_id < 1:
        return {"error": "Product ID must be positive"}
    return {
        "product_id": product_id,
        "name": f"Product {product_id}",
        "price": 99.99
    }

print("✅ Path parameter examples created!")
print("\n📝 Routes:")
print("   - GET /items/{item_id}        (int)")
print("   - GET /users/{username}       (str)")
print("   - GET /users/{user_id}/posts/{post_id}  (multiple params)")
print("   - GET /products/{product_id}  (int with validation)")
print("\n💡 Try accessing:")
print("   - /items/123  ✅ (valid)")
print("   - /items/abc  ❌ (422 error - not an integer)")

## Part 3: Query Parameters

Query parameters are optional values passed in the URL after `?`.

### Syntax

```python
@app.get("/items/")
def list_items(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}
```

### How It Works:
- Parameters with default values become query parameters
- Access via: `/items/?skip=0&limit=10`
- Optional by default (because of default value)
- Required if no default value provided

### Query Parameter Types:

| Type | Example | Description |
|------|---------|-------------|
| **Required** | `q: str` | Must be provided |
| **Optional** | `skip: int = 0` | Has default value |
| **Optional (None)** | `q: str = None` | Can be omitted |
| **Multiple** | `tags: list[str]` | Can have multiple values |

In [ ]:
# Example: Query Parameters

from fastapi import FastAPI
from typing import Optional, List

app = FastAPI()

# Required query parameter
@app.get("/search/")
def search(q: str):
    """
    Search endpoint with required query parameter
    
    Access: /search/?q=fastapi
    """
    return {
        "query": q,
        "results": [f"Result for '{q}'"]
    }

# Optional query parameters with defaults
@app.get("/items/")
def list_items(skip: int = 0, limit: int = 10):
    """
    List items with pagination
    
    Access: /items/?skip=0&limit=10
    Access: /items/?skip=10&limit=20
    Access: /items/  (uses defaults: skip=0, limit=10)
    """
    return {
        "skip": skip,
        "limit": limit,
        "items": [f"Item {i}" for i in range(skip, skip + limit)]
    }

# Optional query parameter (can be None)
@app.get("/users/")
def list_users(
    active: Optional[bool] = None,
    role: Optional[str] = None
):
    """
    List users with optional filters
    
    Access: /users/
    Access: /users/?active=true
    Access: /users/?active=true&role=admin
    """
    filters = {}
    if active is not None:
        filters["active"] = active
    if role is not None:
        filters["role"] = role
    
    return {
        "filters": filters,
        "users": ["User 1", "User 2"]
    }

# Multiple query parameters
@app.get("/products/")
def list_products(
    category: str = "all",
    min_price: float = 0.0,
    max_price: Optional[float] = None,
    sort_by: str = "name",
    order: str = "asc"
):
    """
    List products with multiple query parameters
    
    Access: /products/?category=electronics&min_price=100&max_price=500&sort_by=price&order=desc
    """
    return {
        "category": category,
        "price_range": {
            "min": min_price,
            "max": max_price or "unlimited"
        },
        "sort": {
            "by": sort_by,
            "order": order
        },
        "products": ["Product 1", "Product 2"]
    }

# Boolean query parameters
@app.get("/posts/")
def list_posts(published: bool = True, featured: bool = False):
    """
    Boolean query parameters
    
    Access: /posts/?published=true&featured=false
    Access: /posts/?published=1&featured=0  (also works)
    """
    return {
        "published": published,
        "featured": featured,
        "posts": ["Post 1", "Post 2"]
    }

print("✅ Query parameter examples created!")
print("\n📝 Examples:")
print("   - GET /search/?q=fastapi")
print("   - GET /items/?skip=0&limit=10")
print("   - GET /users/?active=true&role=admin")
print("   - GET /products/?category=electronics&min_price=100")
print("\n💡 Key Points:")
print("   - Parameters with defaults = optional query params")
print("   - Parameters without defaults = required query params")
print("   - Use Optional[Type] = None for truly optional")
print("   - FastAPI automatically converts types")

In [ ]:
# Example: Combining Path and Query Parameters

from fastapi import FastAPI
from typing import Optional

app = FastAPI()

# Path parameter + Query parameters
@app.get("/users/{user_id}/posts")
def get_user_posts(
    user_id: int,           # Path parameter (required)
    skip: int = 0,          # Query parameter (optional)
    limit: int = 10         # Query parameter (optional)
):
    """
    Get posts by user with pagination
    
    Access: /users/123/posts
    Access: /users/123/posts?skip=10&limit=20
    """
    return {
        "user_id": user_id,
        "skip": skip,
        "limit": limit,
        "posts": [f"Post {i} by user {user_id}" for i in range(skip, skip + limit)]
    }

# Multiple path params + query params
@app.get("/users/{user_id}/posts/{post_id}/comments")
def get_post_comments(
    user_id: int,           # Path parameter
    post_id: int,           # Path parameter
    page: int = 1,          # Query parameter
    per_page: int = 10      # Query parameter
):
    """
    Get comments for a specific post
    
    Access: /users/123/posts/456/comments?page=1&per_page=20
    """
    return {
        "user_id": user_id,
        "post_id": post_id,
        "page": page,
        "per_page": per_page,
        "comments": [f"Comment {i} on post {post_id}" for i in range(per_page)]
    }

# Path param with filtering query params
@app.get("/items/{item_id}")
def get_item(
    item_id: int,                    # Path parameter
    include_details: bool = False,   # Query parameter
    format: Optional[str] = None     # Query parameter
):
    """
    Get item with optional details
    
    Access: /items/123
    Access: /items/123?include_details=true
    Access: /items/123?include_details=true&format=json
    """
    item = {
        "item_id": item_id,
        "name": f"Item {item_id}",
        "price": 99.99
    }
    
    if include_details:
        item["details"] = {
            "description": f"Description for item {item_id}",
            "stock": 100
        }
    
    if format:
        item["format"] = format
    
    return item

print("✅ Combined path and query parameters!")
print("\n📝 Examples:")
print("   - GET /users/123/posts")
print("   - GET /users/123/posts?skip=10&limit=20")
print("   - GET /users/123/posts/456/comments?page=2&per_page=15")
print("   - GET /items/123?include_details=true&format=json")
print("\n💡 Remember:")
print("   - Path params: from URL segments (required)")
print("   - Query params: from ?key=value (optional if default)")

## Part 5: Request Body with POST

POST endpoints receive data in the request body to create new resources.

### Basic POST Endpoint

```python
from pydantic import BaseModel

class Item(BaseModel):
    name: str
    price: float

@app.post("/items/")
def create_item(item: Item):
    return item
```

### How It Works:
- ✅ Use `@app.post()` decorator
- ✅ Define Pydantic model for request body
- ✅ FastAPI automatically validates JSON
- ✅ Invalid data returns 422 error
- ✅ Returns 201 Created status code (by default)

### Request Body Types:

| Type | Use Case | Example |
|------|----------|---------|
| **Pydantic Model** | Structured data | `item: Item` |
| **dict** | Simple JSON | `data: dict` |
| **List** | Array of items | `items: List[Item]` |
| **Multiple Models** | Complex requests | `item: Item, user: User` |

In [ ]:
# Example: POST Endpoints with Request Body

from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional, List

app = FastAPI()

# Pydantic model for request body
class ItemCreate(BaseModel):
    """Model for creating an item"""
    name: str
    price: float
    description: Optional[str] = None

# Simple POST endpoint
@app.post("/items/")
def create_item(item: ItemCreate):
    """
    Create a new item
    
    Request body (JSON):
    {
        "name": "Laptop",
        "price": 999.99,
        "description": "Gaming laptop"
    }
    """
    return {
        "message": "Item created",
        "item": item.dict(),
        "id": 123  # Simulated ID
    }

# POST with validation
class UserCreate(BaseModel):
    name: str
    email: str
    age: Optional[int] = None

@app.post("/users/")
def create_user(user: UserCreate):
    """
    Create a new user
    
    FastAPI automatically:
    - Validates required fields (name, email)
    - Converts types
    - Returns 422 if validation fails
    """
    return {
        "message": "User created",
        "user": user.dict(),
        "id": 456
    }

# POST with nested models
class Address(BaseModel):
    street: str
    city: str
    zip_code: str

class UserWithAddress(BaseModel):
    name: str
    email: str
    address: Address

@app.post("/users/with-address/")
def create_user_with_address(user: UserWithAddress):
    """Create user with nested address"""
    return {
        "message": "User with address created",
        "user": user.dict()
    }

# POST with list of items
@app.post("/items/bulk/")
def create_items(items: List[ItemCreate]):
    """
    Create multiple items at once
    
    Request body:
    [
        {"name": "Item 1", "price": 10.0},
        {"name": "Item 2", "price": 20.0}
    ]
    """
    return {
        "message": f"Created {len(items)} items",
        "items": [item.dict() for item in items]
    }

print("✅ POST endpoint examples created!")
print("\n📝 POST Endpoints:")
print("   - POST /items/")
print("   - POST /users/")
print("   - POST /users/with-address/")
print("   - POST /items/bulk/")
print("\n💡 Key Points:")
print("   - Use Pydantic models for validation")
print("   - FastAPI validates automatically")
print("   - Invalid data = 422 error")
print("   - Returns 201 Created by default")

## Part 6: Return Types and Responses

FastAPI uses return types to generate API documentation and validate responses.

### Basic Return Types

```python
@app.get("/items/")
def list_items() -> List[Item]:
    return [Item(name="Item 1", price=10.0)]
```

### Response Types:

| Return Type | Description | Use Case |
|------------|-------------|----------|
| **dict** | Simple JSON object | `{"message": "Hello"}` |
| **Pydantic Model** | Validated response | `Item(name="...", price=10.0)` |
| **List** | Array of items | `[item1, item2]` |
| **Response** | Custom response | Headers, status code, etc. |
| **JSONResponse** | JSON with custom status | `JSONResponse(content={...}, status_code=201)` |

### Status Codes:

| Code | Method | Meaning |
|------|--------|---------|
| **200** | GET, PUT, PATCH | Success |
| **201** | POST | Created |
| **204** | DELETE | No Content |
| **400** | Any | Bad Request |
| **404** | Any | Not Found |
| **422** | Any | Validation Error |

In [ ]:
# Example: Return Types and Responses

from fastapi import FastAPI, Response, status
from fastapi.responses import JSONResponse
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI()

# Pydantic models
class Item(BaseModel):
    id: int
    name: str
    price: float

class ItemResponse(BaseModel):
    """Response model"""
    id: int
    name: str
    price: float
    in_stock: bool = True

# Example 1: Simple dict return
@app.get("/simple/")
def simple_response() -> dict:
    """Returns a simple dictionary"""
    return {"message": "Hello", "status": "ok"}

# Example 2: Return Pydantic model
@app.get("/items/{item_id}", response_model=ItemResponse)
def get_item(item_id: int) -> ItemResponse:
    """
    Get item with response model
    
    response_model ensures:
    - Only specified fields are returned
    - Types are validated
    - API docs show correct schema
    """
    return ItemResponse(
        id=item_id,
        name=f"Item {item_id}",
        price=99.99,
        in_stock=True
    )

# Example 3: Return list with response_model
@app.get("/items/", response_model=List[ItemResponse])
def list_items() -> List[ItemResponse]:
    """List all items"""
    return [
        ItemResponse(id=1, name="Item 1", price=10.0),
        ItemResponse(id=2, name="Item 2", price=20.0)
    ]

# Example 4: Custom status code
@app.post("/items/", status_code=status.HTTP_201_CREATED)
def create_item(item: Item) -> Item:
    """
    Create item with custom status code
    
    Returns 201 Created instead of default 200 OK
    """
    return item

# Example 5: JSONResponse for full control
@app.post("/items/custom/")
def create_item_custom(item: Item):
    """
    Create item with custom response
    
    Full control over status code and headers
    """
    return JSONResponse(
        status_code=201,
        content={
            "message": "Item created successfully",
            "item": item.dict(),
            "location": f"/items/{item.id}"
        },
        headers={"X-Custom-Header": "custom-value"}
    )

# Example 6: Response with headers
@app.get("/items/{item_id}/download")
def download_item(item_id: int):
    """
    Download item (example with custom headers)
    """
    return Response(
        content=f"Item {item_id} data",
        media_type="application/octet-stream",
        headers={
            "Content-Disposition": f"attachment; filename=item_{item_id}.txt"
        }
    )

print("✅ Return type examples created!")
print("\n📝 Response Types:")
print("   - dict: Simple JSON")
print("   - Pydantic Model: Validated response")
print("   - List[Model]: Array of validated items")
print("   - JSONResponse: Full control")
print("   - Response: Custom headers/media types")
print("\n💡 Benefits of response_model:")
print("   - Automatic validation")
print("   - Better API documentation")
print("   - Type safety")
print("   - Filters out extra fields")

## Part 7: Complete CRUD Example

Let's build a complete example with all route types!

In [ ]:
# Complete CRUD Example: Item Management API

from fastapi import FastAPI, HTTPException, status
from pydantic import BaseModel
from typing import List, Optional

app = FastAPI(title="Item Management API")

# Pydantic Models
class ItemCreate(BaseModel):
    """Model for creating an item"""
    name: str
    description: Optional[str] = None
    price: float
    in_stock: bool = True

class ItemUpdate(BaseModel):
    """Model for updating an item (all fields optional)"""
    name: Optional[str] = None
    description: Optional[str] = None
    price: Optional[float] = None
    in_stock: Optional[bool] = None

class ItemResponse(BaseModel):
    """Model for item response"""
    id: int
    name: str
    description: Optional[str]
    price: float
    in_stock: bool

# In-memory database (for demo)
items_db = {}
next_id = 1

# CREATE - POST
@app.post("/items/", response_model=ItemResponse, status_code=status.HTTP_201_CREATED)
def create_item(item: ItemCreate) -> ItemResponse:
    """
    Create a new item
    
    - **name**: Item name (required)
    - **description**: Item description (optional)
    - **price**: Item price (required)
    - **in_stock**: Whether item is in stock (default: True)
    """
    global next_id
    item_id = next_id
    next_id += 1
    
    new_item = ItemResponse(
        id=item_id,
        name=item.name,
        description=item.description,
        price=item.price,
        in_stock=item.in_stock
    )
    items_db[item_id] = new_item
    return new_item

# READ (one) - GET with path parameter
@app.get("/items/{item_id}", response_model=ItemResponse)
def get_item(item_id: int) -> ItemResponse:
    """
    Get a specific item by ID
    
    - **item_id**: The ID of the item to retrieve
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found"
        )
    return items_db[item_id]

# READ (many) - GET with query parameters
@app.get("/items/", response_model=List[ItemResponse])
def list_items(
    skip: int = 0,
    limit: int = 10,
    in_stock: Optional[bool] = None
) -> List[ItemResponse]:
    """
    List all items with pagination and filtering
    
    - **skip**: Number of items to skip (default: 0)
    - **limit**: Maximum number of items to return (default: 10)
    - **in_stock**: Filter by stock status (optional)
    """
    items = list(items_db.values())
    
    # Filter by stock status if provided
    if in_stock is not None:
        items = [item for item in items if item.in_stock == in_stock]
    
    # Apply pagination
    return items[skip:skip + limit]

# UPDATE - PUT (full update)
@app.put("/items/{item_id}", response_model=ItemResponse)
def update_item(item_id: int, item: ItemCreate) -> ItemResponse:
    """
    Update an entire item (replace all fields)
    
    - **item_id**: The ID of the item to update
    - **item**: New item data (all fields required)
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found"
        )
    
    updated_item = ItemResponse(
        id=item_id,
        name=item.name,
        description=item.description,
        price=item.price,
        in_stock=item.in_stock
    )
    items_db[item_id] = updated_item
    return updated_item

# UPDATE - PATCH (partial update)
@app.patch("/items/{item_id}", response_model=ItemResponse)
def patch_item(item_id: int, item: ItemUpdate) -> ItemResponse:
    """
    Partially update an item (only provided fields)
    
    - **item_id**: The ID of the item to update
    - **item**: Fields to update (only provided fields are updated)
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found"
        )
    
    current_item = items_db[item_id]
    update_data = item.dict(exclude_unset=True)  # Only include provided fields
    
    updated_item = ItemResponse(
        id=item_id,
        name=update_data.get("name", current_item.name),
        description=update_data.get("description", current_item.description),
        price=update_data.get("price", current_item.price),
        in_stock=update_data.get("in_stock", current_item.in_stock)
    )
    items_db[item_id] = updated_item
    return updated_item

# DELETE
@app.delete("/items/{item_id}", status_code=status.HTTP_204_NO_CONTENT)
def delete_item(item_id: int):
    """
    Delete an item
    
    - **item_id**: The ID of the item to delete
    """
    if item_id not in items_db:
        raise HTTPException(
            status_code=status.HTTP_404_NOT_FOUND,
            detail=f"Item {item_id} not found"
        )
    del items_db[item_id]
    return None  # 204 No Content

print("✅ Complete CRUD API created!")
print("\n📝 Available Endpoints:")
print("   POST   /items/              - Create item")
print("   GET    /items/{item_id}     - Get item by ID")
print("   GET    /items/              - List items (with pagination)")
print("   PUT    /items/{item_id}     - Update entire item")
print("   PATCH  /items/{item_id}     - Partially update item")
print("   DELETE /items/{item_id}     - Delete item")
print("\n💡 Features demonstrated:")
print("   ✅ Path parameters")
print("   ✅ Query parameters")
print("   ✅ Request body (POST, PUT, PATCH)")
print("   ✅ Response models")
print("   ✅ Status codes")
print("   ✅ Error handling")

In [ ]:
# Example: Advanced Route Patterns

from fastapi import FastAPI, Query, Path, Body
from pydantic import BaseModel, Field
from enum import Enum
from typing import Optional

app = FastAPI()

# Pattern 1: Enum in path parameter
class ItemType(str, Enum):
    """Item type enumeration"""
    electronics = "electronics"
    clothing = "clothing"
    food = "food"
    books = "books"

@app.get("/items/type/{item_type}")
def get_items_by_type(item_type: ItemType):
    """
    Get items by type using enum
    
    Access: /items/type/electronics
    Access: /items/type/clothing
    Invalid: /items/type/invalid  (422 error)
    """
    return {
        "type": item_type,
        "items": [f"{item_type.value} item 1", f"{item_type.value} item 2"]
    }

# Pattern 2: Query parameter with validation
@app.get("/items/")
def list_items(
    skip: int = Query(default=0, ge=0, description="Number of items to skip"),
    limit: int = Query(default=10, ge=1, le=100, description="Max items to return"),
    search: Optional[str] = Query(default=None, min_length=2, max_length=50)
):
    """
    List items with validated query parameters
    
    Query() allows:
    - Validation (ge=greater or equal, le=less or equal)
    - Documentation
    - Default values
    """
    return {
        "skip": skip,
        "limit": limit,
        "search": search,
        "items": ["Item 1", "Item 2"]
    }

# Pattern 3: Path parameter with validation
@app.get("/items/{item_id}")
def get_item(
    item_id: int = Path(..., gt=0, description="Item ID must be positive")
):
    """
    Get item with validated path parameter
    
    Path(...) means required
    gt=0 means greater than 0
    """
    return {"item_id": item_id, "name": f"Item {item_id}"}

# Pattern 4: Pydantic Field validation in models
class ItemCreate(BaseModel):
    name: str = Field(..., min_length=1, max_length=100)
    price: float = Field(..., gt=0, description="Price must be positive")
    description: Optional[str] = Field(None, max_length=500)

@app.post("/items/")
def create_item(item: ItemCreate):
    """Create item with field validation"""
    return {"message": "Item created", "item": item.dict()}

# Pattern 5: Multiple query parameters with Query()
@app.get("/search/")
def search(
    q: str = Query(..., min_length=1, description="Search query"),
    category: Optional[str] = Query(None, regex="^(electronics|clothing|food)$"),
    min_price: Optional[float] = Query(None, ge=0),
    max_price: Optional[float] = Query(None, gt=0)
):
    """
    Search with multiple validated parameters
    
    - q: Required, at least 1 character
    - category: Optional, must match regex pattern
    - min_price: Optional, must be >= 0
    - max_price: Optional, must be > 0
    """
    return {
        "query": q,
        "category": category,
        "price_range": {
            "min": min_price,
            "max": max_price
        },
        "results": ["Result 1", "Result 2"]
    }

print("✅ Advanced route patterns created!")
print("\n📝 Advanced Features:")
print("   - Enum path parameters (type safety)")
print("   - Query() for parameter validation")
print("   - Path() for path parameter validation")
print("   - Field() for model field validation")
print("   - Regex validation")
print("   - Range validation (ge, le, gt, lt)")

## Part 9: Common Patterns and Best Practices

### Best Practice 1: Use Response Models

✅ **Good:**
```python
@app.get("/items/", response_model=List[ItemResponse])
def list_items():
    return items
```

❌ **Bad:**
```python
@app.get("/items/")
def list_items():
    return items  # No type safety
```

### Best Practice 2: Validate Input

✅ **Good:**
```python
@app.get("/items/{item_id}")
def get_item(item_id: int = Path(..., gt=0)):
    ...
```

❌ **Bad:**
```python
@app.get("/items/{item_id}")
def get_item(item_id: int):
    if item_id < 1:  # Manual validation
        ...
```

### Best Practice 3: Use Descriptive Names

✅ **Good:**
```python
@app.get("/users/{user_id}/posts")
def get_user_posts(user_id: int):
    ...
```

❌ **Bad:**
```python
@app.get("/u/{id}/p")
def get_p(id: int):
    ...
```

### Best Practice 4: Handle Errors Properly

✅ **Good:**
```python
if item_id not in items_db:
    raise HTTPException(status_code=404, detail="Item not found")
```

❌ **Bad:**
```python
if item_id not in items_db:
    return {"error": "Not found"}  # Wrong status code
```

In [ ]:
# Example: Best Practices in Action

from fastapi import FastAPI, HTTPException, Query, Path
from pydantic import BaseModel, Field
from typing import List, Optional

app = FastAPI()

class ItemResponse(BaseModel):
    id: int
    name: str
    price: float

# ✅ Good: Response model, validated parameters, proper error handling
@app.get("/items/{item_id}", response_model=ItemResponse)
def get_item(
    item_id: int = Path(..., gt=0, description="Item ID must be positive")
) -> ItemResponse:
    """
    Get item by ID - Best practice example
    
    Features:
    - Response model for type safety
    - Path validation (must be > 0)
    - Proper error handling
    - Type hints everywhere
    """
    items = {1: ItemResponse(id=1, name="Item 1", price=10.0)}
    
    if item_id not in items:
        raise HTTPException(
            status_code=404,
            detail=f"Item {item_id} not found"
        )
    
    return items[item_id]

# ✅ Good: Query parameters with validation
@app.get("/items/", response_model=List[ItemResponse])
def list_items(
    skip: int = Query(0, ge=0, description="Items to skip"),
    limit: int = Query(10, ge=1, le=100, description="Max items to return")
) -> List[ItemResponse]:
    """
    List items - Best practice example
    
    Features:
    - Validated query parameters
    - Response model
    - Clear documentation
    """
    all_items = [
        ItemResponse(id=1, name="Item 1", price=10.0),
        ItemResponse(id=2, name="Item 2", price=20.0),
        ItemResponse(id=3, name="Item 3", price=30.0)
    ]
    return all_items[skip:skip + limit]

print("✅ Best practices demonstrated!")
print("\n💡 Key Takeaways:")
print("   ✅ Always use response_model")
print("   ✅ Validate parameters with Query()/Path()")
print("   ✅ Use HTTPException for errors")
print("   ✅ Add type hints everywhere")
print("   ✅ Write clear docstrings")

## Part 10: Route Order and Precedence

### Important: Route Order Matters!

FastAPI matches routes in the order they're defined. More specific routes should come first.

### Example:

```python
# ✅ Good: Specific route first
@app.get("/users/me")
def get_current_user():
    return {"user": "current"}

@app.get("/users/{user_id}")
def get_user(user_id: int):
    return {"user_id": user_id}
```

❌ **Bad:**
```python
# Generic route first - "me" would be treated as user_id!
@app.get("/users/{user_id}")
def get_user(user_id: int):
    ...

@app.get("/users/me")
def get_current_user():
    ...
```

### Route Matching Rules:

1. **Exact matches first** - `/users/me` before `/users/{user_id}`
2. **More specific paths first** - `/users/{user_id}/posts/{post_id}` before `/users/{user_id}`
3. **Query parameters don't affect matching** - Only path matters

In [ ]:
# Example: Route Order and Precedence

from fastapi import FastAPI

app = FastAPI()

# ✅ Correct order: Specific routes first
@app.get("/users/me")
def get_current_user():
    """Get current authenticated user - specific route"""
    return {"user": "current", "id": "me"}

@app.get("/users/admin")
def get_admin():
    """Get admin user - specific route"""
    return {"user": "admin", "role": "administrator"}

@app.get("/users/{user_id}")
def get_user(user_id: int):
    """Get user by ID - generic route (comes after specific)"""
    return {"user_id": user_id, "name": f"User {user_id}"}

# ✅ Correct: More specific nested routes first
@app.get("/users/{user_id}/posts/recent")
def get_recent_posts(user_id: int):
    """Get recent posts - specific route"""
    return {"user_id": user_id, "posts": "recent"}

@app.get("/users/{user_id}/posts/{post_id}")
def get_user_post(user_id: int, post_id: int):
    """Get specific post - generic route"""
    return {"user_id": user_id, "post_id": post_id}

print("✅ Route order examples!")
print("\n📝 Route Matching Order:")
print("   1. /users/me              (exact match)")
print("   2. /users/admin           (exact match)")
print("   3. /users/{user_id}       (path parameter)")
print("   4. /users/{user_id}/posts/recent  (specific)")
print("   5. /users/{user_id}/posts/{post_id}  (generic)")
print("\n💡 Remember:")
print("   - Specific routes MUST come before generic ones")
print("   - FastAPI matches in definition order")
print("   - Wrong order = routes won't work as expected")

## Part 11: Summary Table - Route Types

### Quick Reference

| Route Type | Decorator | Parameters | Use Case |
|------------|-----------|------------|----------|
| **Simple GET** | `@app.get("/")` | None | Root endpoint, simple data |
| **Path Param** | `@app.get("/items/{id}")` | `id: int` | Get specific resource |
| **Query Param** | `@app.get("/items/")` | `skip: int = 0` | Filtering, pagination |
| **POST Body** | `@app.post("/items/")` | `item: ItemCreate` | Create new resource |
| **PUT Body** | `@app.put("/items/{id}")` | `id: int, item: Item` | Replace entire resource |
| **PATCH Body** | `@app.patch("/items/{id}")` | `id: int, item: ItemUpdate` | Partial update |
| **DELETE** | `@app.delete("/items/{id}")` | `id: int` | Remove resource |

### Parameter Types

| Type | Syntax | Example | Required? |
|------|--------|---------|-----------|
| **Path** | `{param}` in URL | `/items/{item_id}` | ✅ Always |
| **Query** | Function parameter | `skip: int = 0` | ❌ If has default |
| **Body** | Pydantic model | `item: ItemCreate` | ✅ Always |
| **Optional Query** | `Optional[Type] = None` | `q: Optional[str] = None` | ❌ No |

In [ ]:
# Quick Reference: All Route Types

from fastapi import FastAPI
from pydantic import BaseModel
from typing import Optional

app = FastAPI()

class ItemCreate(BaseModel):
    name: str
    price: float

# 1. Simple GET
@app.get("/")
def root():
    return {"message": "Hello"}

# 2. GET with path parameter
@app.get("/items/{item_id}")
def get_item(item_id: int):
    return {"item_id": item_id}

# 3. GET with query parameters
@app.get("/items/")
def list_items(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

# 4. GET with path + query
@app.get("/users/{user_id}/posts")
def get_posts(user_id: int, page: int = 1):
    return {"user_id": user_id, "page": page}

# 5. POST with request body
@app.post("/items/")
def create_item(item: ItemCreate):
    return {"item": item.dict()}

# 6. PUT with path + body
@app.put("/items/{item_id}")
def update_item(item_id: int, item: ItemCreate):
    return {"item_id": item_id, "item": item.dict()}

# 7. PATCH with path + body
@app.patch("/items/{item_id}")
def patch_item(item_id: int, item: ItemCreate):
    return {"item_id": item_id, "item": item.dict()}

# 8. DELETE with path parameter
@app.delete("/items/{item_id}")
def delete_item(item_id: int):
    return {"message": f"Item {item_id} deleted"}

print("✅ All route types demonstrated!")
print("\n📋 Route Type Cheat Sheet:")
print("   GET    /items/{id}        → Path parameter")
print("   GET    /items/?skip=0     → Query parameter")
print("   POST   /items/            → Request body")
print("   PUT    /items/{id}        → Path + body")
print("   PATCH  /items/{id}        → Path + body (partial)")
print("   DELETE /items/{id}        → Path parameter")

## Part 12: Key Takeaways & Summary

### What You've Learned

✅ **GET Endpoints** - Retrieve data with `@app.get()`  
✅ **Path Parameters** - Dynamic URL segments `{param}`  
✅ **Query Parameters** - Optional URL parameters `?key=value`  
✅ **POST Request Body** - Send data with Pydantic models  
✅ **Return Types** - Control responses with `response_model`  
✅ **Status Codes** - Set appropriate HTTP status codes  
✅ **Error Handling** - Use `HTTPException` for errors  
✅ **Validation** - Automatic validation with type hints  

### Key Concepts

1. **Path Parameters**
   - From URL segments: `/items/{item_id}`
   - Always required
   - Type hints enable validation

2. **Query Parameters**
   - From URL query string: `?skip=0&limit=10`
   - Optional if they have default values
   - Use `Query()` for advanced validation

3. **Request Body**
   - Use Pydantic models
   - Automatic JSON validation
   - Invalid data = 422 error

4. **Response Types**
   - Use `response_model` for validation
   - Better API documentation
   - Type safety

### Best Practices

1. ✅ Use `response_model` for all endpoints
2. ✅ Validate parameters with `Query()` and `Path()`
3. ✅ Use descriptive function and parameter names
4. ✅ Handle errors with `HTTPException`
5. ✅ Put specific routes before generic ones
6. ✅ Use type hints everywhere
7. ✅ Write clear docstrings

---

## 🎉 Congratulations!

You've mastered FastAPI basic route handling!

**You now know:**
- How to create GET, POST, PUT, PATCH, DELETE endpoints
- How to use path and query parameters
- How to handle request bodies
- How to control responses and status codes
- Best practices for route organization

**You're ready to build complete REST APIs with FastAPI!** 🚀

### Next Steps:
1. Practice building more complex endpoints
2. Learn about dependency injection
3. Explore advanced Pydantic features
4. Build a complete CRUD API
5. Add authentication and authorization

Keep practicing and building! 💪